In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
%load_ext autoreload
%autoreload 2

In [ ]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

# Load data

## Manually Labeled Trajectories

In [ ]:
# Load data into pandas
expert = pd.read_csv("/content/drive/MyDrive/datasets/manual_trajectories/expert.csv")
pgy4 = pd.read_csv("/content/drive/MyDrive/datasets/manual_trajectories/pgy4.csv")
pgy2 = pd.read_csv("/content/drive/MyDrive/datasets/manual_trajectories/pgy2.csv")
trajectories = [expert, pgy4, pgy2]

In [ ]:
expert

## Keypoint Generated Paths

In [ ]:
def load_path(filename, header=0):
  '''Reads the coordinates from a path file into a dataframe.

  Frames where a forceps was not detected are filtered out.

  Args:
    filename: The path filepath to load from
  
  Returns:
    A pandas dataframe of the coordinates where the forceps is present, sorted
      by frame
  '''
  data = pd.read_csv(filename, header=header)
  return data[data.key_L_x.notnull()].sort_values(by=['frame_num'])

In [ ]:
def file_label(filename):
  '''Determines the label for data from a file based on the filename.

  Args:
    filename: The path filepath to determine based on

  Returns:
    A number representing the true class of the data:
      0 = Junior Resident
      1 = Senior Resident
      2 = Expert
  '''
  if filename.startswith("Medi"):
    return 0
  elif filename.startswith(("KY", "AC")):
    return 1
  elif filename.startswith(("Cataract", "SQ")):
    return 2
  else:
    raise Exception("Unhandled filetype: " + filename)

In [ ]:
data = load_path("/content/drive/MyDrive/Projects/rhexis-trajectory/Trajectory_Classification/out.csv")
data

In [ ]:
def get_video_resolution(video_resolution_df, filename):
  col = video_resolution_df[filename]
  return (col[1], col[2])

In [ ]:
VIDEO_SIZE_LOC = "/content/drive/MyDrive/Rhexis/datasets/test_pulls/pull_info.csv"
video_size_df = pd.read_csv(VIDEO_SIZE_LOC)
get_video_resolution(video_size_df, "CataractCoach1_0_63")

In [ ]:
def convert_path_to_img_matrix(path_df, fill_val_w_frame = False):
  '''Converts a list of path coordinates to an image matrix.
  In order to handle complexity with floats, the size of the matrix is 10x the
  number of pixels, and each point is rounded to the nearest one-tenth pixel.

  The matrix is also bounded such that the bottom-left pixel in the trajectory
  is placed at (0, 0), by shifting all points by the minimum coordinate.

  Additionally, there are two modes in which the matrix can be filled:
    1. Boolean path matrix: Places a "1" in each position the forceps was
         at any point in the trajectory
    2. Frame number matrix: Places the frame number in which the forceps was
         at a location. This may help with incorporate temporal quality.

  Args:
    path_df: the path coordinate dataframe to convert
    fill_val_w_frame: Whether to fill in the frame number as the coordinate
      value. Defaults to False.

  Returns;
    The 2D np array representing the trajectory
  '''
  lf_coords = path_df[["key_L_x", "key_L_y", "frame_num"]]
  lf_coords["key_L_x"] = lf_coords["key_L_x"].subtract(lf_coords["key_L_x"].min()) 
  lf_coords["key_L_y"] = lf_coords["key_L_y"].subtract(lf_coords["key_L_y"].min())
  lf_coords = np.rint(lf_coords * 10).astype(int)

  maxes = lf_coords.max()
  arr = np.zeros((maxes[0], maxes[1]))
  lf_np = lf_coords.to_numpy()
  arr[lf_np[:, 0]-1, lf_np[:, 1]-1] = lf_np[:, 2] if fill_val_w_frame else 1
  return arr

In [ ]:
def convert_path_to_std_pupil_matrix(path_df, size=None, fill_val_w_frame=False, verbose=False):
  '''Maps a path to a normalized matrix representing position relative to the
  center of a pupil.

  Args:
    path_df: The path coordinate dataframe to convert
    vid_res: The resolution of the video the path was taken from, which helps
      determine if any clipping of the pupil has occurred
    size: If set, maps the pupil to a specified size matrix. If unset, uses
      the pupil height and width for the matrix.
    fill_val_w_frame: Whether to fill in the frame number as the coordinate
      value. Defaults to False.
  '''
  # Determine the pupil size initial matrix
  pupil_width = abs(path_df["pupil_right_x"] - path_df["pupil_left_x"]).mean()
  pupil_height = abs(path_df["pupil_up_y"] - path_df["pupil_down_y"]).mean()
  arr = np.zeros(size or (pupil_width, pupil_height))
  arr_center_x, arr_center_y = (np.array(arr.shape) / 2).astype(int)
  # vid_width, vid_height = vid_res
  # return center_x, center_y

  for i in range(len(path_df)):
    # Calculate how much to translate forceps point to center pupil at center
    # of matrix
    # pupil_center_x = path_df["pupil_center_x"].iloc[i]
    # pupil_center_y = path_df["pupil_center_y"].iloc[i]
    # translation_factor_x = arr_center_x - pupil_center_x 
    # translation_factor_y = arr_center_y - pupil_center_y
    translation_factor_x = path_df["pupil_left_x"].iloc[i]
    translation_factor_y = path_df["pupil_up_y"].iloc[i]
    if verbose:
      print("=======================================================")
      print("Translate: " + str((translation_factor_x, translation_factor_y)))

    # Calculate scale factor for forceps
    # Assume elliptical if tilted. Find max between both sides to ensure one side
    # is not clipped
    pcx = path_df["pupil_center_x"].iloc[i]
    pcy = path_df["pupil_center_y"].iloc[i]
    plx = path_df["pupil_left_x"].iloc[i]
    prx = path_df["pupil_right_x"].iloc[i]
    puy = path_df["pupil_up_y"].iloc[i]
    pdy = path_df["pupil_down_y"].iloc[i]
     
    # if pdy == vid_height or puy == 0:
    #   radius = np.max(np.array([pcx - plx, prx - pcx]))
    # elif prx == vid_width or plx == 0:
    #   radius = np.max(np.array([pcy - puy, pdy - pcy]))
    # else:
    #   radius = np.max(np.array([pcy - puy, pdy - pcy, pcx - plx, prx - pcx]))
    # left_radius = pcx - plx
    # right_radius = prx - pcx
    radius_x = max(pcx - plx, prx - pcx)
    radius_y = max(pcy - puy, pdy - pcy)
    # radius_x = abs(plx - prx) if (prx == vid_width or plx == 0) else (left_radius+right_radius)/2
    # up_radius = pcy - puy
    # down_radius = pdy - pcy
    # radius_y = abs(puy - pdy) if (pdy == vid_height or puy == 0) else (up_radius+down_radius)/2
    # radius_x = max(path_df["pupil_center_x"].iloc[i] - path_df["pupil_left_x"].iloc[i],
    #                path_df["pupil_right_x"].iloc[i] - path_df["pupil_center_x"].iloc[i])
    # radius_y = max(path_df["pupil_center_y"].iloc[i] - path_df["pupil_up_y"].iloc[i],
    #                path_df["pupil_down_y"].iloc[i] - path_df["pupil_center_y"].iloc[i])
    scale_factor_x = arr_center_x / radius_x
    scale_factor_y = arr_center_y / radius_y
    # scale_factor_x = arr_center_x / radius
    # scale_factor_y = arr_center_y / radius
    if verbose:
      print("Scale: " + str((scale_factor_x, scale_factor_y)))

    coord_l_x = int((path_df["key_L_x"].iloc[i] - translation_factor_x) * scale_factor_x)
    coord_l_y = int((path_df["key_L_y"].iloc[i] - translation_factor_y) * scale_factor_y)

    if verbose:
      print("Coord: " + str((coord_l_y, coord_l_x)))
      print("=======================================================")

    if coord_l_x > 0 and coord_l_y > 0:
      arr[coord_l_y, coord_l_x] = path_df["frame_num"].iloc[i] if fill_val_w_frame*10 else 1
  return arr



In [ ]:
def get_pupil_std_data(matrix_size=None, fill_val_w_frame=False):
  '''Loads data and performs conversion of paths to standardized pupil centered
  matrices.

  Args:
    matrix_size: The matrix size to project onto for the conversion function
    fill_val_w_frame: Whether to use the frame number as the value in the matrix.
      Defaults to 1.
  
  Returns:
    Tuple of matrix list and corresponding labels
  '''
  matrix_size = matrix_size or (100, 100)
  DATA_LOC = "/content/drive/MyDrive/Rhexis/datasets/test_pulls/OUTPUT/"
  files = os.listdir(DATA_LOC)
  # VIDEO_SIZE_LOC = "/content/drive/MyDrive/Rhexis/datasets/test_pulls/pull_info.csv"
  # video_size_df = pd.read_csv(VIDEO_SIZE_LOC)
  path_dfs = [load_path(f'{DATA_LOC}/{file}') for file in files]
  # path_vid_sizes = [get_video_resolution(video_size_df, file.split('_fea')[0]) for file in files]
  labels = [file_label(file) for file in files]
  # path_matrices = [convert_path_to_std_pupil_matrix(df, res, matrix_size, fill_val_w_frame) for df, res in zip(path_dfs, path_vid_sizes)]
  path_matrices = [convert_path_to_std_pupil_matrix(df, matrix_size, fill_val_w_frame) for df in path_dfs]
  return np.stack(path_matrices, axis=0), np.array(labels)

In [ ]:
def get_pupil_std_data_traj(matrix_size=None, fill_val_w_frame=False):
  '''Loads data and performs conversion of paths to standardized pupil centered
  matrices.

  Args:
    matrix_size: The matrix size to project onto for the conversion function
    fill_val_w_frame: Whether to use the frame number as the value in the matrix.
      Defaults to 1.
  
  Returns:
    Tuple of matrix list and corresponding labels
  '''
  matrix_size = matrix_size or (100, 100)
  DATA_LOC = "/content/drive/MyDrive/Rhexis/datasets/test_pulls/OUTPUT_TRAJECTORIES/"
  files = os.listdir(DATA_LOC)
  # VIDEO_SIZE_LOC = "/content/drive/MyDrive/Rhexis/datasets/test_pulls/pull_info.csv"
  # video_size_df = pd.read_csv(VIDEO_SIZE_LOC)
  path_dfs = [load_path(f'{DATA_LOC}/{file}', header=2) for file in files]
  # path_vid_sizes = [get_video_resolution(video_size_df, file.split('_fea')[0]) for file in files]
  labels = [file_label(file) for file in files]
  path_matrices = [convert_path_to_std_pupil_matrix(df, matrix_size, fill_val_w_frame) for df in path_dfs]
  return np.stack(path_matrices, axis=0), np.array(labels)

In [ ]:
def pad_matrices(path_matrices):
  '''Pads the right and up sides of a matrix with 0s to make all matrices in a
  list the same shape. Preserves the property that the bottom-left most point
  in the trajectory will be at (0,0)

  Args:
    path_matrices: List of matrices to pad
  
  Returns:
    A list of matrices of the same size
  '''
  max_x = max(map(lambda x: x.shape[0], path_matrices))
  max_y = max(map(lambda x: x.shape[1], path_matrices))
  for i, m in enumerate(path_matrices):
    path_matrices[i] = np.pad(m, ((0, max_x-m.shape[0]), (0, max_y-m.shape[1])), mode='constant')
    print(path_matrices[i].shape)
  return path_matrices

In [ ]:
# plt.scatter(nonnull_data.pupil_center_x, nonnull_data.pupil_center_y, c="red")
# plt.scatter(nonnull_data.pupil_right_x, nonnull_data.pupil_center_y, c="blue")
# plt.scatter(nonnull_data.pupil_left_x, nonnull_data.pupil_center_y, c="purple")
# plt.scatter(nonnull_data.pupil_center_x, nonnull_data.pupil_up_y, c="teal")
# plt.scatter(nonnull_data.pupil_center_x, nonnull_data.pupil_down_y, c="pink")
# plt.scatter(nonnull_data.key_R_x, nonnull_data.key_R_y, c="green")
# plt.scatter(nonnull_data.key_L_x, nonnull_data.key_L_y, c="yellow")

# Exploratory Visualization

In [ ]:
X, y = get_pupil_std_data_traj((100, 100), False)
for m in X:
  plt.imshow(m)
  plt.show()

## (OLD) Full Trajectories

In [ ]:
# Plot PGY2 trajectory
for i in range(max(pgy2['pull']) + 1):
  plt.scatter(pgy2[pgy2['pull'] == i]['x'], pgy2[pgy2['pull'] == i]['y'])

In [ ]:
# Plot PGY4 trajectory
for i in range(max(pgy4['pull']) + 1):
  plt.scatter(pgy4[pgy4['pull'] == i]['x'], pgy4[pgy4['pull'] == i]['y'])

In [ ]:
# Plot expert trajectory
for i in range(max(expert['pull']) + 1):
  plt.scatter(expert[expert['pull'] == i]['x'], expert[expert['pull'] == i]['y'])

## (OLD) Individual Pulls

In [ ]:
# Plot all 4 pulls in expert trajectory
for i in range(max(expert['pull']) + 1):
  plt.figure()
  plt.scatter(expert[expert['pull'] == i]['x'], expert[expert['pull'] == i]['y'])
plt.show()

In [ ]:
# Plot all PGY4 trajectories
for i in range(max(pgy4['pull']) + 1):
  plt.figure()
  plt.scatter(pgy4[pgy4['pull'] == i]['x'], pgy4[pgy4['pull'] == i]['y'])
plt.show()

In [ ]:
# Plot all PGY2 trajectories
for i in range(max(pgy2['pull']) + 1):
  plt.figure()
  plt.scatter(pgy2[pgy2['pull'] == i]['x'], pgy2[pgy2['pull'] == i]['y'])
plt.show()

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
def pca(n_components):
  pca = PCA(n_components=n_components, svd_solver="randomized", whiten=True).fit(X_train)


# Feature Extraction Functions

In [ ]:
def pull_to_length(pull):
    """
    Args:
      pull: The dataframe for the pull to conver to velocities
  
    Return:
      velocities: velocities calculated as distance traveled per frame
    """
    return len(pull.x)

In [ ]:
def pull_to_velocities_and_accelerations(pull):
  """
  Args:
    pull: The dataframe for the pull to conver to velocities
  
  Return:
    mean velocity: velocity calculated as distance traveled per frame
    mean acceleration: acceleration calculated as change in velocity between frames
  """
  x, y = pull.x, pull.y
  velocities = []
  accelerations = []

  for i in range(len(x)-1):
    x1, x2 = x[i:i+2]
    y1, y2 = y[i:i+2]
    velocities.append(np.sqrt(np.square(x2 - x1) + np.square(y2 - y1)))

  for i in range(len(velocities) - 1):
    accelerations.append(np.abs(velocities[i+1] - velocities[i]))

  return (np.mean(velocities), np.mean(accelerations))

In [ ]:
def pull_to_angles(pull):
  """
  Args:
    pull: The dataframe for the pull to convert to angles

  Return:
    angles: The angles for each triple of datapoints
  """
  x, y = pull.x, pull.y
  angles = []
  for i in range(len(x)-2):
    x1, x2, x3 = x[i:i+3]
    y1, y2, y3 = y[i:i+3]
    v1 = np.array([x1-x2, y1-y2])
    v2 = np.array([x3-x2, y3-y2])
    angle_nocos = np.dot(v1, v2) / (np.linalg.norm(v1, 2) * np.linalg.norm(v2, 2))
    angle_floor = np.where(angle_nocos < -1, -1.0, angle_nocos)
    angle_ceil = np.where(angle_floor > 1, 1.0, angle_floor) 
    angle = np.arccos(angle_ceil) * 180 / np.pi
    angles.append(angle)
  return angles

In [ ]:
def angles_to_bins(angles, num_bins=20):
  """
  Args:
    angles: the list of angles to bin

  Returns:
    histogram: histogram values
    bins: histogram bins
  """
  bins = [i for i in range(0, 181, 180//num_bins)] 
  return np.histogram(angles, bins)


In [ ]:
def featurize_pull(pull):
  """
  Performs all featurizations for an individual pull

  Args:
    pull: The pull dataframe to convert to a feature row

  Returns:
    row: A feature row for the pull
  """
  angles = pull_to_angles(pull)
  hist, bins = angles_to_bins(angles)

  length = pull_to_length(pull)
  mean_velocity, mean_accel = pull_to_velocities_and_accelerations(pull)

  features = np.append(hist, np.array([length, mean_velocity, mean_accel]))

  return features

In [ ]:
def featurize_trajectory(trajectory, label, transform_func=None):
  """
  Splits a trajectory by pulls and featurizes each pull. Each pull is assigned
  the input label.

  Args:
    trajectory: The trajectory dataframe to split into featurized pulls
    label: The label to assign each pull for the trajectory
    transform_func: Custom transformation to use during featurization. Defaults
        to `featurize_pull`.
  
  Return:
    X: The feature rows
    y: The labels for each row
  """
  transform_func = transform_func or featurize_pull
  print(transform_func)
  X = [transform_func(df) for val, df in trajectory.groupby('pull')]
  y = np.repeat(label, len(X))
  return np.array(X), y

In [ ]:
def featurize_trajectories(trajectories, transform_func=None):
  """
  Converts a list of trajectories into a feature matrix and label vector

  Args:
    trajectory: The trajctories list to split into featurized pulls
    transform_func: Custom transformation to use during featurization. Defaults
        to `featurize_pull`.
  
  Return:
    X: The feature matrix
    y: The label vector
  """
  featurized_trajectory_list = [featurize_trajectory(t, i, transform_func) for i, t in enumerate(trajectories)]
  X = np.row_stack([x for x, y in featurized_trajectory_list])
  y = np.concatenate([y for x, y in featurized_trajectory_list])
  return X, y

In [ ]:
def extract_pulls(trajectories):
  """
  Returns raw pulls by modifying featurization method to apply identity function

  Args:
    trajectories: The trajctories dataframe to extract pulls from
  """
  return featurize_trajectories(trajectories, lambda x: x)

# Model Fitting/Prediction

In [ ]:
X, y = get_pupil_std_data_traj((100, 100), False)

In [ ]:
np.sum(X < 0)

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

# Keep deterministic
np.random.seed(42)
sss = StratifiedShuffleSplit(1, test_size=.2)
train_ind, test_ind = next(sss.split(X, y))
X_train, X_test = X[train_ind], X[test_ind]
y_train, y_test = y[train_ind], y[test_ind]
# Print to check class balance
y_train, y_test

## PCA

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
class ImageFlatteningTransformer(BaseEstimator, TransformerMixin):
  def __init__(self):
    return
  def fit(self, X, y=None):
    return self
  def transform(self, X, y=None):
    return np.reshape(X, (-1, 10000))
  
def process_for_pca(data):
  '''TODO
  '''
  return np.reshape(data, (-1, 10000))

In [ ]:
from sklearn.decomposition import PCA, KernelPCA
def pca(data, n_components):
  pca = PCA(n_components=n_components, svd_solver="randomized", whiten=True).fit(data)
  return pca, pca.components_.reshape((n_components, 100, 100))

In [ ]:
pca_obj, components = pca(process_for_pca(X_train), 4)
# pca_obj.explained_variance_
X_train_pca = pca_obj.transform(process_for_pca(X_train))
for i in components:
  plt.imshow(i)
  plt.show()

In [ ]:
# components = pca(process_for_pca(X_train), 5)
# for i in range(len(components)):
#   plt.imshow(components[i])
#   plt.show()

## Logistic Model (Baseline)

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
def make_custom_pipeline(clf, n_components):
  return make_pipeline(ImageFlatteningTransformer(), 
                       StandardScaler(),
                       PCA(n_components=n_components, svd_solver="randomized", whiten=True),
                       clf)
  
def make_custom_kernel_pca_pipeline(clf, n_components=None):
  return make_pipeline(ImageFlatteningTransformer(), 
                       StandardScaler(),
                       KernelPCA(n_components=n_components, kernel="rbf", gamma=None, fit_inverse_transform=True, alpha=0.1),
                       clf)


In [ ]:
X_train[0].shape

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

clf = LogisticRegression(random_state=0, multi_class="multinomial", max_iter=1000)
pipe = make_custom_pipeline(clf, 10)
# cross_val_score(pipe, X_train_pca, y_train, cv=5, scoring='recall_macro')
pipe.fit(X_train, y_train)
cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy')

In [ ]:
clf = LogisticRegression(random_state=0, multi_class="multinomial", max_iter=1000)
pipe = make_custom_kernel_pca_pipeline(clf)
pipe.fit(X_train, y_train)
cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy')

In [ ]:
flattened_train = ImageFlatteningTransformer().transform(X_train)
flattened_test = ImageFlatteningTransformer().transform(X_test)
scaled_train = StandardScaler().fit_transform(flattened_train)
scaled_test = StandardScaler().fit(flattened_train).fit_transform(flattened_test)
kpca = KernelPCA(n_components=None, kernel="rbf", gamma=None, fit_inverse_transform=True, alpha=0.1)
pcad = kpca.fit(scaled_train).transform(scaled_test)
pcad
plt.scatter(pcad[:, 3], pcad[:, 1])
# np.mean(scaled)
# np.var(scaled, axis=0)[np.var(scaled, axis=0) != 0]
# np.mean(StandardScaler().fit_transform(flattened_train))
# np.var(StandardScaler().fit_transform(flattened_train))


In [ ]:
components = pipe.steps[2][1].components_
for component in components:
  im = component.reshape(100, 100)
  plt.imshow(im)
  plt.show()

In [ ]:
pipe.score(X_test, y_test)

## Quadratic GDA

This does not perform well

In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

clf = QuadraticDiscriminantAnalysis()
cross_val_score(clf, X_train_pca, y_train, cv=5, scoring='recall_macro')
# clf.fit(X_train, y_train)
# clf.score(X_test, y_test)

In [ ]:
clf = QuadraticDiscriminantAnalysis
pipe = make_custom_kernel_pca_pipeline(clf)
print(pipe)
# pipe.fit(X_train, y_train)
cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy')

## Unsupervised clustering

In [ ]:
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import accuracy_score

BATCH_SIZE = 3
kmeans = MiniBatchKMeans(n_clusters=3,
                          random_state=0,
                          batch_size=BATCH_SIZE)
for i in range(0, len(X_train), BATCH_SIZE):
  kmeans.partial_fit(X_train[i:i+BATCH_SIZE])

y_pred = kmeans.predict(X_test)
accuracy_score(y_test, y_pred)

## Neural Network

In [ ]:
from sklearn.neural_network import MLPClassifier


clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                     hidden_layer_sizes=(1000, 100), random_state=1)
pipe = make_pipeline(ImageFlatteningTransformer(), 
                     StandardScaler(),
                     clf)
pipe.fit(X_train, y_train)
cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy')